# Blind Spot Detection (BSD) with Active Intervention

*Project for the course Smart Vehicular Systems (Academic Year 2024/2025).*

**Name:** Pablo Sebastian

**Surname:** Vargas Grateron

**Email:** pablo.vargasgrateron@studio.unibo.it

This project implements a blind spot detection system using [CARLA simulator](https://carla.org/) (Version 0.9.15) and MQTT for communication. The system detects vehicles in the blind spot of a simulated car and can trigger an active intervention if necessary.

> Before running the code, ensure that the CARLA server is running and the MQTT broker is accessible.

## Libraries and setup

To execute the code, you need to install the libraries listed in the `requirements.txt` file. You can install them using:
```bash
pip install -r requirements.txt
```

For the local imports, ensure that the `manual_control.py` and `manual_control_steeringwheel.py` files are in the same directory as this notebook or adjust the import path accordingly.

> The local imports can be found in the CARLA repository under the `PythonAPI/examples` directory.

In [1]:
import carla, time, pygame, math, random, cv2, os
import numpy as np
import paho.mqtt.client as mqtt
from ultralytics import YOLO

# Local imports
import manual_control
import manual_control_steeringwheel

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


Set the following boolean to `False` if you want to use the keyboard control instead of the steering wheel, or set it to `True` if you want to use the steering wheel control.
Set the `CAR_MODEL` to the desired vehicle model you want to use in the simulation. You can find a list of available vehicle models in [this list](https://github.com/carla-simulator/carla/blob/ue5-dev/Docs/catalogue_vehicles.md#audi-tt).

> Remember to set the `wheel_config.ini` file in the same directory as this notebook to configure the steering wheel settings.

In [2]:
IS_STEERING_WHEEL = False
CAR_MODEL = "vehicle.audi.tt"

This cell configures the CARLA server connection.

In [3]:
CARLA_HOST = 'localhost'
CARLA_PORT = 2000
CARLA_TIMEOUT = 10.0

carla_client = carla.Client(CARLA_HOST, CARLA_PORT)
carla_client.set_timeout(CARLA_TIMEOUT)

world = carla_client.get_world()
spectator = world.get_spectator()

This cell configures the MQTT client. We will use the public broker `test.mosquitto.org` for testing purposes. In a production environment, you should use a secure and private MQTT broker.

In [4]:
MQTT_BROKER = "test.mosquitto.org"
MQTT_PORT = 1883
MQTT_TOPIC = "svs_blind_spot_detection"

mqtt_client = mqtt.Client()
mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)

c:\Users\pseba\.conda\envs\carla-env\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  """


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

## Functions

### CARLA basic functions (From the laboratory)

In [5]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

### MQTT functions

Further setup for the MQTT client, including connection and message handling.

In [6]:
def mqtt_on_connect(client, userdata, flags, rc):
    """
    Callback function triggered when MQTT client connects to broker
    - Automatically subscribes to the BSD topic once connected
    - rc codes: 0=success, 1=wrong protocol, 2=invalid client ID, etc.
    """
    print(f"INFO: [MQTT] Connected (code {rc})")
    client.subscribe(MQTT_TOPIC)

def mqtt_on_message(client, userdata, message):
    """
    Callback function triggered when a message is received on subscribed topics
    - Decodes incoming messages and prints them
    """
    decoded_message = message.payload.decode()
    print(f"INFO: [MQTT] Message on {message.topic} -> {decoded_message}")

def mqtt_send_message(topic, message):
    """
    Send a message to specified MQTT topic
    - Used to publish BSD alerts, status updates, or intervention commands
    - Message format: string (will be encoded to bytes automatically)
    """
    mqtt_client.publish(topic, message)
    print(f"INFO: [MQTT] Sent message: {message} to topic: {topic}")

mqtt_client.on_connect = mqtt_on_connect
mqtt_client.on_message = mqtt_on_message    

mqtt_client.loop_start()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

### Computer vision functions

We will implement functions to capture images from the car's cameras and detect vehicles using [YOLOv5](https://docs.ultralytics.com/models/yolov5/) model. We will use the nano version for real-time processing.

> Make sure to download the YOLOv5 nano model and place it in the `cv_model` directory (Or change the path in the code below).

In [7]:
MODEL_DIR = os.path.join(os.getcwd(), "cv_model")
MODEL_NAME = "yolov5nu.pt"

cv_model_path = os.path.join(MODEL_DIR, MODEL_NAME)
cv_model = YOLO(cv_model_path)

def detect_vehicles(image, model=cv_model, conf=0.25):
    """
    Detect vehicles (car, truck, bus, motorcycle) in an image using YOLOv5.

    Args:
        image (numpy.ndarray): Input image in RGB format.
        model: YOLOv5 model.
        conf (float): Confidence threshold (default: 0.25).

    Returns:
        list of dict: Detected vehicles with bounding box and class info.
    """
    results = model(image)
    
    # Results in tensor [x1, y1, x2, y2, conf, class]
    predictions = results.xyxy[0].cpu().numpy()
    detections = []

    for pred in predictions:
        x1, y1, x2, y2, confidence, class_id = pred
        if confidence >= conf:
            label = model.names[int(class_id)]
            if label in ['car', 'truck', 'bus', 'motorcycle']:
                detections.append({
                    'xmin': int(x1),
                    'ymin': int(y1),
                    'xmax': int(x2),
                    'ymax': int(y2),
                    'confidence': float(confidence),
                    'class_id': int(class_id),
                    'label': label
                })

    return detections

def carla_image_to_array(image):
    """
    Convert CARLA raw image data to NumPy RGB array for YOLO processing
    
    Args:
        image: CARLA camera sensor image object with raw_data attribute
    
    Returns:
        numpy.ndarray: RGB image array (height, width, 3) suitable for YOLO
    """
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3]
    array = array[:, :, [2, 1, 0]] # Convert BGR to RGB
    return array

def process_camera_image(image):
    """
    Complete pipeline: Convert CARLA image and detect vehicles for BSD system
    
    Args:
        image: CARLA camera sensor image object
    
    Returns:
        tuple: (rgb_array, detections)
            - rgb_array: Converted RGB image as NumPy array
            - detections: List of vehicle detections from YOLO
    """
    rgb_array = carla_image_to_array(image)
    detections = detect_vehicles(rgb_array)
    
    if detections:
        print(f"Detected {len(detections)} vehicles:")
        for det in detections:
            print(f"  - {det['label']}: {det['confidence']:.2f} confidence")
    
    return rgb_array, detections

def detect_car_in_sensor(image):
    """
    Detect cars in the given sensor image using YOLOv5.

    Args:
        image: CARLA camera sensor image object.

    Returns:
        list: Detected car bounding boxes and labels.
    """
    rgb_array, detections = process_camera_image(image)
    
    if not detections:
        print("[CAR DETECTION] No cars detected.")
        return False
    if detections:
        print(f"[CAR DETECTION] Detected {len(detections)} cars.")
        for det in detections:
            if det['confidence'] > 0.5:
                print(f"[CAR DETECTION] Car detected: {det['label']} with confidence {det['confidence']:.2f}")
                return True
    return False

### Sensors functions

The next cells implement the functions to spawn:

- The **camera sensors** to capture image to further process with YOLOv5 and detect vehicles.
- The **radar sensor** to detect vehicles in the blind spot.

In [8]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=0.0, y=0.0, z=1.0), carla.Rotation(pitch=0)), width=800, height=600):
    """
    Spawn an RGB camera sensor in the CARLA world for vehicle detection
    
    Args:
        attach_to: Vehicle or object to attach camera to (None = spawn freely)
        transform: Camera position and orientation relative to attachment point
        width: Image width in pixels (default: 800)
        height: Image height in pixels (default: 600)
    
    Returns:
        carla.Sensor: RGB camera sensor object for capturing images
    """
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x=0.0, y=0.0, z=1.0), carla.Rotation(pitch=0)), range=100, horizontal_fov=30, vertical_fov=30):
    """
    Spawn a radar sensor in the CARLA world for object detection
    
    Args:
        attach_to: Vehicle or object to attach radar to (None = spawn freely)
        transform: Radar position and orientation relative to attachment point
        range: Detection range in meters (default: 100m)
        horizontal_fov: Horizontal field of view in degrees (default: 30°)
        vertical_fov: Vertical field of view in degrees (default: 30°)
    
    Returns:
        carla.Sensor: Radar sensor object for detecting nearby objects
    """
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('range', str(range))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    return radar

### Debug functions

The following cells contain debug functions to visualize different sensors data inside the CARLA simulator.

#### Sensor placement check

In [ ]:
def debug_draw_sensor_placement(vehicle, camera_left, camera_right, radar_left_side, radar_right_side, radar_left_rear, radar_right_rear):
    """
    Draw sensor placements on the CARLA world for debugging
    
    Args:
        vehicle: Vehicle actor to draw sensors on
        camera_left: Left camera sensor actor
        camera_right: Right camera sensor actor
        radar_left_side: Left side radar sensor actor
        radar_right_side: Right side radar sensor actor
        radar_left_rear: Left rear radar sensor actor
        radar_right_rear: Right rear radar sensor actor
    """
    draw_on_screen(world, camera_left.get_transform(), content='Camera Left', color=carla.Color(0, 255, 0))
    draw_on_screen(world, camera_right.get_transform(), content='Camera Right', color=carla.Color(0, 255, 0))
    draw_on_screen(world, radar_left_side.get_transform(), content='Radar Left Side', color=carla.Color(255, 0, 0))
    draw_on_screen(world, radar_right_side.get_transform(), content='Radar Right Side', color=carla.Color(255, 0, 0))
    draw_on_screen(world, radar_left_rear.get_transform(), content='Radar Left Rear', color=carla.Color(255, 0, 0))
    draw_on_screen(world, radar_right_rear.get_transform(), content='Radar Right Rear', color=carla.Color(255, 0, 0))
    move_spectator_to(vehicle.get_transform(), distance=10, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0)

#### Radar cone visualization

In [ ]:
def rotate_point(x, y, z, rotation):
    """
    Rotate point (x, y, z) by CARLA rotation (pitch, yaw, roll) in degrees.
    """
    # Convert degrees to radians
    pitch = math.radians(rotation.pitch)
    yaw = math.radians(rotation.yaw)
    roll = math.radians(rotation.roll)

    # Apply yaw (Z-axis)
    x_yaw = x * math.cos(yaw) - y * math.sin(yaw)
    y_yaw = x * math.sin(yaw) + y * math.cos(yaw)
    z_yaw = z

    # Apply pitch (Y-axis)
    x_pitch = x_yaw * math.cos(pitch) + z_yaw * math.sin(pitch)
    y_pitch = y_yaw
    z_pitch = -x_yaw * math.sin(pitch) + z_yaw * math.cos(pitch)

    # Apply roll (X-axis)
    x_roll = x_pitch
    y_roll = y_pitch * math.cos(roll) - z_pitch * math.sin(roll)
    z_roll = y_pitch * math.sin(roll) + z_pitch * math.cos(roll)

    return x_roll, y_roll, z_roll


def debug_draw_cone(actor, horizontal_fov=30.0, vertical_fov=10.0, range_m=20.0):
    """
    Draw the radar's field of view cone considering its orientation.
    """
    world = actor.get_world()
    transform = actor.get_transform()
    radar_location = transform.location
    radar_rotation = transform.rotation

    # Half-angles in radians
    h_fov = math.radians(horizontal_fov / 2)
    v_fov = math.radians(vertical_fov / 2)

    # Local points (relative to radar)
    local_points = []
    for yaw in [-h_fov, h_fov]:
        for pitch in [-v_fov, v_fov]:
            x = range_m * math.cos(pitch) * math.cos(yaw)
            y = range_m * math.cos(pitch) * math.sin(yaw)
            z = range_m * math.sin(pitch)
            local_points.append((x, y, z))

    # Transform local points to world coordinates
    world_points = []
    for (x, y, z) in local_points:
        x_r, y_r, z_r = rotate_point(x, y, z, radar_rotation)
        world_points.append(radar_location + carla.Location(x=x_r, y=y_r, z=z_r))

    # Draw lines from radar to each corner
    for p in world_points:
        world.debug.draw_line(radar_location, p, thickness=0.05, color=carla.Color(0, 255, 0), life_time=20)

    # Draw the rectangle connecting the far points
    world.debug.draw_line(world_points[0], world_points[1], thickness=0.05, color=carla.Color(0, 255, 0), life_time=20)
    world.debug.draw_line(world_points[1], world_points[3], thickness=0.05, color=carla.Color(0, 255, 0), life_time=20)
    world.debug.draw_line(world_points[3], world_points[2], thickness=0.05, color=carla.Color(0, 255, 0), life_time=20)
    world.debug.draw_line(world_points[2], world_points[0], thickness=0.05, color=carla.Color(0, 255, 0), life_time=20)

## Main loop code

### Debug flag

Debug flags to enable or disable debug features. These features are useful for visualizing sensor placements and radar cones in the CARLA simulator and are purely for debugging purposes.

> **WARNING:** Is recommended to set only one debug flag to `True` at a time to avoid overlapping tags, drawings and misplacement of the cameras.

In [ ]:
DEBUG_DRAW_SENSOR_PLACEMENT = False
DEBUG_DRAW_RADAR_CONE = False

### Sensor parameters

The next cell contains the parameters of the sensors. These parameters define the pitch, yaw and FOV of the sensors, as well as the range of the radar sensors. The parameters are used to spawn the sensors in the CARLA simulator.

In [ ]:
# Pitch and yaw angles for the cameras and radars

CAMERA_LEFT_PITCH = 0
CAMERA_LEFT_YAW = -150
CAMERA_RIGHT_PITCH = 0
CAMERA_RIGHT_YAW = 150

RADAR_SIDE_LEFT_PITCH = 0
RADAR_SIDE_LEFT_YAW = -100
RADAR_SIDE_RIGHT_PITCH = 0
RADAR_SIDE_RIGHT_YAW = 100

RADAR_REAR_LEFT_PITCH = 0
RADAR_REAR_LEFT_YAW = -165
RADAR_REAR_RIGHT_PITCH = 0
RADAR_REAR_RIGHT_YAW = 165

# FOV of the radars

RADAR_SIDE_HORIZONTAL_FOV = 100.0
RADAR_SIDE_VERTICAL_FOV = 15.0
RADAR_SIDE_RANGE = 6.0

RADAR_REAR_HORIZONTAL_FOV = 30.0
RADAR_REAR_VERTICAL_FOV = 10.0
RADAR_REAR_RANGE = 10.0

# Speed THRESHOLD for rear radars
REAR_RADAR_SPEED_THRESHOLD = 0.5

### Car setup

The following cell contains the setup for the car and its sensors. It includes the car's position, orientation, and the sensors to be used for blind spot detection.

The cameras and the radar sensors on the sides are atached near the side mirrors of the car, while the rear sensors are placed at the back of the car, near the rear lights.

In [12]:
def setup_player_vehicle(vehicle):
    """
    Setup sensors for the player vehicle in the CARLA world.

    Args:
        vehicle: The vehicle actor to attach sensors to.

    Returns:
        tuple: Contains the camera and radar sensors attached to the vehicle.
    """
    camera_left = spawn_camera(attach_to=vehicle, transform=carla.Transform(carla.Location(x=0.3, y=-1.0, z=0.5), 
                                                                                carla.Rotation(pitch=CAMERA_LEFT_PITCH, yaw=CAMERA_LEFT_YAW, roll=0)))
    camera_right = spawn_camera(attach_to=vehicle, transform=carla.Transform(carla.Location(x=0.3, y=1.0, z=0.5), 
                                                                                carla.Rotation(pitch=CAMERA_RIGHT_PITCH, yaw=CAMERA_RIGHT_YAW, roll=0)))

    radar_left_side = spawn_radar(attach_to=vehicle, transform=carla.Transform(carla.Location(x=-1.0, y=-1.0, z=0.5), 
                                                                                carla.Rotation(pitch=RADAR_SIDE_LEFT_PITCH, yaw=RADAR_SIDE_LEFT_YAW, roll=0)), 
                                                                                range=RADAR_SIDE_RANGE, horizontal_fov=RADAR_SIDE_HORIZONTAL_FOV, vertical_fov=RADAR_SIDE_VERTICAL_FOV)
    radar_right_side = spawn_radar(attach_to=vehicle, transform=carla.Transform(carla.Location(x=-1.0, y=1.0, z=0.5), 
                                                                                carla.Rotation(pitch=RADAR_SIDE_RIGHT_PITCH, yaw=RADAR_SIDE_RIGHT_YAW, roll=0)), 
                                                                                range=RADAR_SIDE_RANGE, horizontal_fov=RADAR_SIDE_HORIZONTAL_FOV, vertical_fov=RADAR_SIDE_VERTICAL_FOV)

    radar_left_rear = spawn_radar(attach_to=vehicle, transform=carla.Transform(carla.Location(x=-2.0, y=-1.0, z=0.5), 
                                                                                carla.Rotation(pitch=RADAR_REAR_LEFT_PITCH, yaw=RADAR_REAR_LEFT_YAW, roll=0)), 
                                                                                range=RADAR_REAR_RANGE, horizontal_fov=RADAR_REAR_HORIZONTAL_FOV, vertical_fov=RADAR_REAR_VERTICAL_FOV)
    radar_right_rear = spawn_radar(attach_to=vehicle, transform=carla.Transform(carla.Location(x=-2.0, y=1.0, z=0.5), 
                                                                                carla.Rotation(pitch=RADAR_REAR_RIGHT_PITCH, yaw=RADAR_REAR_RIGHT_YAW, roll=0)), 
                                                                                range=RADAR_REAR_RANGE, horizontal_fov=RADAR_REAR_HORIZONTAL_FOV, vertical_fov=RADAR_REAR_VERTICAL_FOV)

    return (camera_left, camera_right, radar_left_side, radar_right_side, radar_left_rear, radar_right_rear)

### Listeners setup

The following cell sets up the listeners for the sensors. The listeners are used to receive the data from the sensors and process it.

Depending on the type of sensor, the data is processed differently. For example, the camera sensors capture images and process them with YOLOv5 to detect vehicles, while the radar sensors detect vehicles in the blind spot.

While the side radars tell if there is a vehicle in the blind spot by only checking the presence of a vehicle, the rear radars also check the speed of the vehicle to determine if it is approaching or moving away from the car.

In [ ]:
def radar_side_callback(data):
    """
    Callback function for radar sensor data.
    - Processes radar data to detect objects in the blind spot.
    """
    for detection in data:
        if detection.depth < RADAR_SIDE_RANGE:
            # TODO: Queue detection data for further processing
            pass  
    

def radar_rear_callback(data):
    """
    Callback function for rear radar sensor data.
    - Processes radar data to detect moving objects in the rear blind spot.
    """
    for detection in data:
        if detection.depth < RADAR_REAR_RANGE and detection.velocity > REAR_RADAR_SPEED_THRESHOLD:
            # TODO: Queue detection data for further processing
            pass

def camera_callback(data):
    """
    Callback function for camera sensor data.
    """
    is_car_detected = detect_car_in_sensor(data)
    if is_car_detected:
        # TODO: Queue detection data for further processing
        pass
    else:
        # TODO: Queue detection data for further processing
        pass

    
    

### Game loop

The following cell contains the game loop and the main function to run the CARLA simulation. It initializes the world, spawns the car, and starts the sensors.

> Most of the code is extracted from the `manual_control.py` and `manual_control_steeringwheel.py` files from the CARLA repository. The code is adapted to this project needs.

In [ ]:
def game_loop(args, client=None, sim_world=None):
    pygame.init()
    pygame.font.init()
    world = None
    original_settings = None

    try:
        if client is None:
            client = carla.Client(args.host, args.port)
            client.set_timeout(10.0)
        else:
            manual_control.logging.info("[GAMELOOP] -> Using existing CARLA client instance")

        if sim_world is None:
            sim_world = client.get_world()
        else:
            manual_control.logging.info("[GAMELOOP] -> Using existing CARLA world instance")

        if args.sync:
            original_settings = sim_world.get_settings()
            settings = sim_world.get_settings()
            if not settings.synchronous_mode:
                settings.synchronous_mode = True
                settings.fixed_delta_seconds = 0.05
            sim_world.apply_settings(settings)

            traffic_manager = client.get_trafficmanager()
            traffic_manager.set_synchronous_mode(True)

        if args.autopilot and not sim_world.get_settings().synchronous_mode:
            manual_control.logging.info("[GAMELOOP] -> WARNING: You are currently in asynchronous mode and could "
                  "experience some issues with the traffic simulation")

        display = pygame.display.set_mode(
            (args.width, args.height),
            pygame.HWSURFACE | pygame.DOUBLEBUF)
        display.fill((0,0,0))
        pygame.display.flip()

        hud = manual_control.HUD(args.width, args.height)
        world = manual_control.World(sim_world, hud, args)

        # Get the first vehicle in the world and setup the sensors
        player_vehicle = sim_world.get_actors().filter('vehicle.*')[0]
        camera_left, camera_right, radar_left_side, radar_right_side, radar_left_rear, radar_right_rear = setup_player_vehicle(player_vehicle)
        manual_control.logging.info("[GAMELOOP] -> Vehicle sensors setup complete")
        
        # [DEBUG] Draw position of sensors
        if DEBUG_DRAW_SENSOR_PLACEMENT:
            debug_draw_sensor_placement(player_vehicle, camera_left, camera_right, radar_left_side, radar_right_side, radar_left_rear, radar_right_rear)
            move_spectator_to(player_vehicle.get_transform(), distance=10, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0)

        # [DEBUG] Draw radar cones
        if DEBUG_DRAW_RADAR_CONE:
            debug_draw_cone(radar_left_side, horizontal_fov=RADAR_SIDE_HORIZONTAL_FOV, vertical_fov=RADAR_SIDE_VERTICAL_FOV, range_m=RADAR_SIDE_RANGE)
            debug_draw_cone(radar_right_side, horizontal_fov=RADAR_SIDE_HORIZONTAL_FOV, vertical_fov=RADAR_SIDE_VERTICAL_FOV, range_m=RADAR_SIDE_RANGE)
            debug_draw_cone(radar_left_rear, horizontal_fov=RADAR_REAR_HORIZONTAL_FOV, vertical_fov=RADAR_REAR_VERTICAL_FOV, range_m=RADAR_REAR_RANGE)
            debug_draw_cone(radar_right_rear, horizontal_fov=RADAR_REAR_HORIZONTAL_FOV, vertical_fov=RADAR_REAR_VERTICAL_FOV, range_m=RADAR_REAR_RANGE)
            move_spectator_to(player_vehicle.get_transform(), distance=20, x=0, y=0, z=6, yaw=0, pitch=-40, roll=0)

        # Register camera and radar callbacks
        camera_left.listen(lambda data: camera_callback(data))
        camera_right.listen(lambda data: camera_callback(data))
        radar_left_side.listen(lambda data: radar_side_callback(data))
        radar_right_side.listen(lambda data: radar_side_callback(data))
        radar_left_rear.listen(lambda data: radar_rear_callback(data))
        radar_right_rear.listen(lambda data: radar_rear_callback(data))
        manual_control.logging.info("[GAMELOOP] -> Sensors registered and listening for data")
        
        # Create controller based on user input
        controller = None
        if args.steering_wheel:
            controller = manual_control_steeringwheel.DualControl(world, args.autopilot)
            manual_control.logging.info("[GAMELOOP] -> Controller initialized with STEERING WHEEL support")
        else:
            controller = manual_control.KeyboardControl(world, args.autopilot)
            manual_control.logging.info("[GAMELOOP] -> Controller initialized with KEYBOARD support")

        if args.sync:
            sim_world.tick()
        else:
            sim_world.wait_for_tick()

        ## ========================================
        ## ======== GAME LOOP  STARTS HERE ========
        ## ========================================

        car_warn_left = False
        car_warn_right = False

        clock = pygame.time.Clock()
        manual_control.logging.info("[GAMELOOP] -> Starting loop...")
        while True:
            if args.sync:
                sim_world.tick()
            clock.tick_busy_loop(args.fps)
            if controller.parse_events(client, world, clock, args.sync):
                return

            # BLIND SPOT DETECTION: Check if there is any vehicle in the blind spot and allert the user by message
            current_light = player_vehicle.get_light_state()
            if not((current_light & carla.VehicleLightState.LeftBlinker) and (current_light & carla.VehicleLightState.RightBlinker)):

                # LEFT blinker check
                if current_light & carla.VehicleLightState.LeftBlinker:
                    car_warn_left = True # TODO: Implement logic for left blinker from sensors
                    if car_warn_left:
                        manual_control.logging.info("[GAMELOOP | TRIGGER] -> Car detected on the LEFT side")
                        mqtt_send_message(MQTT_TOPIC, "[WARNING] Car detected on the LEFT side")
                    break

                # RIGHT blinker check
                if current_light & carla.VehicleLightState.RightBlinker:
                    car_warn_right = True # TODO: Implement logic for right blinker from sensors
                    if car_warn_right:
                        manual_control.logging.info("[GAMELOOP | TRIGGER] -> Car detected on the RIGHT side")
                        mqtt_send_message(MQTT_TOPIC, "[WARNING] -> Car detected on the RIGHT side")
                    break

            # ACTIVE INTERVENTION: If a car is detected and the user tries to change lane, make intervention
            if car_warn_left or car_warn_right:
                # TODO: Check the user movement and make intervention
                pass

            car_warn_left = False
            car_warn_right = False

            world.tick(clock)
            world.render(display)
            pygame.display.flip()
 
    finally:

        if original_settings:
            sim_world.apply_settings(original_settings)

        if (world and world.recording_enabled):
            client.stop_recorder()

        if world is not None:
            world.destroy()

        pygame.quit()


This cell contains the configuration for the game loop, including argument parsing and logging setup.

> To configure the game loop, you can modify the `gameSetup` function to include additional parameters or change the default values.

In [15]:
def game_setup(client=None, sim_world=None):
    argparser = manual_control.argparse.ArgumentParser(
        description='CARLA Manual Control Client')
    argparser.add_argument(
        '-v', '--verbose',
        action='store_true',
        dest='debug',
        help='print debug information')
    argparser.add_argument(
        '--host',
        metavar='H',
        default='127.0.0.1',
        help='IP of the host server (default: 127.0.0.1)')
    argparser.add_argument(
        '-p', '--port',
        metavar='P',
        default=2000,
        type=int,
        help='TCP port to listen to (default: 2000)')
    argparser.add_argument(
        '-a', '--autopilot',
        action='store_true',
        help='enable autopilot')
    argparser.add_argument(
        '--res',
        metavar='WIDTHxHEIGHT',
        default='1280x720',
        help='window resolution (default: 1280x720)')
    argparser.add_argument(
        '--filter',
        metavar='PATTERN',
        default='vehicle.*',
        help='actor filter (default: "vehicle.*")')
    argparser.add_argument(
        '--generation',
        metavar='G',
        default='2',
        help='restrict to certain actor generation (values: "1","2","All" - default: "2")')
    argparser.add_argument(
        '--rolename',
        metavar='NAME',
        default='hero',
        help='actor role name (default: "hero")')
    argparser.add_argument(
        '--gamma',
        default=2.2,
        type=float,
        help='Gamma correction of the camera (default: 2.2)')
    argparser.add_argument(
        '--sync',
        action='store_true',
        help='Activate synchronous mode execution')
    args = argparser.parse_args()

    args.width, args.height = [int(x) for x in args.res.split('x')]

    args.fps = 60

    args.steering_wheel = IS_STEERING_WHEEL

    args.filter = CAR_MODEL

    log_level = manual_control.logging.DEBUG if args.debug else manual_control.logging.INFO
    manual_control.logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)

    manual_control.logging.info('[LOG] -> Listening to server %s:%s', args.host, args.port)

    print(__doc__)

    try:
        game_loop(args, client=client, sim_world=sim_world)

    except KeyboardInterrupt:
        manual_control.logging.info('[GAMELOOP] -> Cancelled by user. Bye!')

### Execution

In [ ]:
game_setup(client=carla_client, sim_world=world)

INFO: [LOG] -> Listening to server 127.0.0.1:2000
INFO: [GAMELOOP] -> Using existing CARLA client instance
INFO: [GAMELOOP] -> Using existing CARLA world instance


Automatically created module for IPython interactive environment


INFO: [GAMELOOP] -> Vehicle sensors setup complete
INFO: [GAMELOOP] -> Sensors registered and listening for data
INFO: [GAMELOOP] -> Controller initialized with KEYBOARD support
INFO: [GAMELOOP] -> Starting loop...


INFO: [MQTT] Connected (code 0)
Left turn signal is ON
Left turn signal is OFF
Right turn signal is ON
Right turn signal is OFF


: 